### Librerias

In [1]:
import time
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import re
from datetime import datetime
from selenium import webdriver
import pprint
import pandas as pd


### Código  

In [2]:
class PaginaScrap:
    """
    Es el Scrap de una Página de 123SEguros.
    ...

    Attributes
    ----------
    companias : lista
        companias en la pagina web
    data : dataframe
        representa la data scrapeada


    Methods
    -------
    guardar_csv(self, ruta="./generica.csv"):
        Prints the person's name and age.
    """
    
    def __init__(self, _id, nombre_excel, mostrar=False):
        self.driver = None
        self.coberturas = ["Responsabilidad Civil","Terceros Completos",\
                           "Terceros Completos Full", "Terceros Completos Full + Granizo",\
                           "Todo Riesgo"]
        
        self.nombre_excel = nombre_excel
        self.url = None
        self._id = _id
        self.companias = [] 
        self.data = None
        self.html = None
        self.soup = None
        self.mostrar = mostrar
        self.date = datetime.date(datetime.now()).strftime("%d/%m/%Y")
        
        #Funciones que arrancan con inicializacion
        self.get_url()
        self.get_selenium_starting()
        self.go_to_url()
        time.sleep(30)
        self.get_companias()
        self.crear_df()
       
    
    def get_selenium_starting(self):
        '''Opciones básicas de Selenium'''
        options = Options()
        options.add_argument("--window-size=1980,1080")
        options.add_argument("start-maximized")
        options.add_experimental_option("excludeSwitches", ["enable-automation"])
        options.add_experimental_option('useAutomationExtension', False)
        options.add_argument("--headless")
        self.driver = webdriver.Chrome(options=options)
    
    
    def get_url(self):
        urls = pd.read_excel(f'./urls.xlsx' , sheet_name="urls")
        url = urls[urls["Id"]==self._id]["Seguros"]
        url = url.to_list()[0]
        self.url = url
        
        
    def go_to_url(self):
        '''Carga el url de la página'''
        self.driver.get(self.url)

        
    def get_companias(self): 
        '''Encuentra las Companias en base a las imagenes'''
        self.html = self.driver.page_source
        self.soup = BeautifulSoup(self.html, 'lxml')
        
        images = self.soup.find_all('img',{'src':re.compile('/images/front/table/*')})
        for image in images:
            compania = image['src'].split("/")[4].split(".")[0] 
            self.companias.append(compania)

        self.companias = self.companias[1:]
    
    def crear_df(self):
        '''Crea el DataFrame con las datas en self.data'''
        rows = [self.soup.find('tr', {"data-company":  compania.capitalize()}) for compania in  self.companias]
        self.data = {}
        for row, compania in zip(rows, self.companias):
            self.data[f"{compania}"] = {} 
            for cob in self.coberturas:
                self.data[f"{compania}"][f"{cob}"] = "nan"
                price_tag = row.find('td', {"data-cobertura":cob}).find("div", {"class":"inner"}) 
                if price_tag is not None:
                    price_string = price_tag.get_text()   
                    self.data[f"{compania}"][f"{cob}"] =  float(price_string.split()[1])
        
        self.data = pd.DataFrame(self.data)
        self.data.insert(0, 'Fecha', self.date)
        self.data = self.data.reset_index()
        self.data.rename(columns={"index":"cobertura"}, inplace=True)   
        self.data.insert(0, "id", self._id)

        return self.data
    
    

    def guardar_excel(self):
        try: 
            completo_df = pd.read_excel(f'./{self.nombre_excel}.xlsx' )
        except:
            completo_df = pd.DataFrame(columns=['id','cobertura','Fecha','allianz','zurich','sura',
                                 'mercantil','provincia','orbis','mapfre','sancor','prudencia','galeno','experta'])
        resultado = pd.concat([completo_df ,self.data,])
        resultado = resultado.drop_duplicates(subset =["cobertura", "id", "Fecha"] ,keep='last')
        resultado = resultado.fillna("nan")
        resultado.to_excel(f'./{self.nombre_excel}.xlsx'  , sheet_name="data", index=False)

In [3]:
tiempo_x_run = 37

def generete_all_data(ids, nombre_excel="data_perfiles"):
    '''Genera todos los datos para la lista de ids que se le pase,
    usando la Clase PaginaScarp'''
    print(f"Tiempo estimado total es {round((tiempo_x_run*len(ids))/60)} minutos.")
    for _id in ids:
        try:
            pagina = PaginaScrap(_id, nombre_excel)
        except:
            print(f"Hubo un erro con el {_id}")
            continue
        pagina.guardar_excel()
        print(f"*ID: {_id}: finalizado correctamente!")
    print("Proceso Completado! ")
        

### Generar todos Los Perfiles 

In [4]:
generete_all_data([1,2,3,4,5])

Tiempo estimado total es 3 minutos.
*ID: 1: finalizado correctamente!
*ID: 2: finalizado correctamente!
*ID: 3: finalizado correctamente!
*ID: 4: finalizado correctamente!
*ID: 5: finalizado correctamente!
Proceso Completado! 


### Generar un único Perfil 

In [39]:
#Ingrese el ID del Perfil
pagina = PaginaScrap(3, nombre_excel="data_perfiles")
pagina.guardar_excel()